In [ ]:
# Example 4: Demodulation of satellite data
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import gpslib_tutorial as gpslib
from scipy.fft import fft, ifft
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

# --- you can change the following variables ----

satNo = 24                   # possible are: 24,15,12,19,13,17,10,23,22,25,32
codePhase = 1370             # put in the measured value from the correlation 
                             # for the given satellite
SHOW_PHASE_PLOT = False
REBUILD_PHASE = False

# ----------------------------------------------

codeSamples = 2048
NCopies = 128
NGPS = NCopies*codeSamples
SAMPLE_RATE = 2.048e6                                            # per second 

SEC_TIME = np.linspace(1,NGPS,NGPS,endpoint=True,
                       dtype=np.complex64)/SAMPLE_RATE           # in s

DATA_FILE = '../data/230914_data_24s.bin'

DF = {24:-667.1,15:-4030.0,12:+1675.0,19:-235.0,13:-4598.0,17:-2319.0,
      10:-1262.0,23:-3345.0,22:-4026.0,25:+2852.0,32:+2648.0 }
dopplerFreq = DF[satNo]
dopplerPhase = 0

# -----------------------------------------------

def readData(dataFile,ns,smpNo):
    i = 0
    with open(dataFile,'rb') as file:
        while i < smpNo:
            byteData = np.fromfile(file,dtype=np.uint16,count=ns)
            im,re = np.divmod(byteData,256)
            data = np.asarray(re+1j*im,dtype=np.complex64)/127.5 - (1+1j)            
            i += 1
    return data

def fftCorrelate(data,cacode): 
    fftData = fft(data)    
    fftCacode = fft(cacode)
    fftCorr = fftData*np.conjugate(fftCacode) 
    corr = ifft(fftCorr)
    return corr

def demodDoppler(data,freq,phase,N):                          
    factor = np.exp(-1j*(2*np.pi*freq*SEC_TIME[:N]+phase/180*np.pi))
    return factor*data[:N]

def calcCorr(freq):
    satData = demodDoppler(RAWDATA,freq,0,codeSamples) 
    corr = fftCorrelate(satData,cacodes[:codeSamples])
    return abs(corr)

def decodeData(delay,freq,phase):
    satData = demodDoppler(RAWDATA,freq,phase,NGPS)
    data = np.roll(satData,-delay)
    signal = data*cacodes
    res = []
    for i in range(NCopies-1):
        m = np.mean(signal[i*codeSamples:(i+1)*codeSamples])
        res.append(m)
    res = np.asarray(res,dtype=np.complex64)
    return res

def rebuildPhase(phase):
    minDiff = 2.0
    dp = 0
    n = len(phase)
    realPhase = np.copy(phase)
    for i in range(1,n):                         
        delta = phase[i]-phase[i-1]
        dp -= np.sign(delta) if abs(delta) > minDiff else 0            
        realPhase[i] += dp*np.pi
    return realPhase

def calcPhase(signal):
    signalPhase = np.arctan(signal.imag/signal.real)
    if REBUILD_PHASE:
        signalPhase = rebuildPhase(signalPhase)
    return signalPhase    

SMP_NO = 2
RAWDATA = readData(DATA_FILE,NGPS,SMP_NO)
cacodes = gpslib.GPSCacodeRep(satNo,codeSamples,NCopies,0)        

# --- code for plots ----------

df_min = dopplerFreq - 50
df_max = dopplerFreq + 50

cp_min = codePhase - 20
cp_max = codePhase + 20

fig = plt.figure(figsize=(13,5))
fig.canvas.header_visible = False      # hide header
fig.subplots_adjust(left=0.05)         #, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
spec = fig.add_gridspec(1,3)
ax1 = fig.add_subplot(spec[0, 0])
ax2 = fig.add_subplot(spec[0, 1])

if SHOW_PHASE_PLOT:
    ax3 = fig.add_subplot(spec[0, 2])

line1, = ax1.plot(calcCorr(dopplerFreq))
ax1.set_xlabel('Delay $\\tau$')
#ax1.set_ylabel('Correlation')
ax1.set_title('Correlation PRN%02d' % (satNo))
ax1.set_ylim(0,100)
#ax1.set_xlim(codePhase-100,codePhase+100)

signal = decodeData(codePhase,dopplerFreq,dopplerPhase)
line2, = ax2.plot(signal.real,label='I(t)')
line3, = ax2.plot(signal.imag,label='Q(t)')
ax2.legend()

ax2.set_xlabel('time / ms')
#ax2.set_ylabel('Amplitude')
ax2.set_title('Amplitude PRN%02d' % (satNo))
ax2.set_ylim(-0.06,+0.06)

if SHOW_PHASE_PLOT:
    signalPhase = calcPhase(signal) 
    line4, = ax3.plot(signalPhase/np.pi*180)
    ax3.set_xlabel('time / ms')
    ax3.set_title('Phase PRN%02d' % (satNo))
    ax3.set_ylim(-95,95)
    
    
# adjust the main plot to make room for the sliders
fig.subplots_adjust(bottom=0.35)

axFreq = fig.add_axes([0.15, 0.2, 0.5, 0.03])
freq_slider = Slider(
    ax=axFreq,
    label='freq/Hz',
    valmin=df_min,
    valmax=df_max,
    valinit=dopplerFreq,
)
    
axPhase = fig.add_axes([0.15, 0.1, 0.5, 0.03])
phase_slider = Slider(
    ax=axPhase,
    label='phase offset ',
    valmin=0,
    valmax=360,
    valinit=dopplerPhase,
)

axDelay = fig.add_axes([0.15, 0.0, 0.5, 0.03])
delay_slider = Slider(
    ax=axDelay,
    label='code phase ',
    valmin=cp_min,
    valmax=cp_max,
    valstep=1,
    valinit=codePhase,
)

# The function to be called anytime a slider's value changes
def update(val):
    line1.set_ydata(calcCorr(freq_slider.val)) 
    signal = decodeData(int(delay_slider.val),freq_slider.val,phase_slider.val)
    line2.set_ydata(signal.real)
    line3.set_ydata(signal.imag)
    if SHOW_PHASE_PLOT:
        signalPhase = calcPhase(signal)
        line4.set_ydata(signalPhase/np.pi*180)            
    fig.canvas.draw_idle()    

freq_slider.on_changed(update)
delay_slider.on_changed(update)
phase_slider.on_changed(update)

plt.show()